## Merge Raw Petfinder Data
207 Final Project<br>
Lucy Herr<br>
7/13/23<br>
### Process Overview
1. Parse Pet Listing Image Metadata
2. Parse Pet Listing Sentiment Analysis Data
3. Merge Image & Sentiment Data with Tabular Data
4. Add external data features (State Population & Median Income)

### Import Packages

In [29]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import json
import flatten_json
import os
import warnings
import wget
import urllib3
from sklearn.model_selection import train_test_split # to split the data

# import cleaning_functions
import cleaning_functions # assumes cleaning functions file is in same working directory as this notebook

### Load data & functions

In [30]:
# data directory path 
data_dir = './data/' # assumes data subdirectory (follows structure of original file)

# load raw tabular training data 
train_file = 'train/train.csv' # assumes data & train subdirectories (follows structure of original file)
train_tab_df = pd.read_csv(data_dir+train_file)

# load raw image metadata (JSON)
image_metadata_path = 'train_metadata/' 

# load raw sentiment metadata (JSON)
sentiment_path = 'train_sentiment/'
sentiment_json_files = [pos_json for pos_json in os.listdir(data_dir+sentiment_path) 
                        if pos_json.endswith('.json')]

# Malaysian state labels 
state_file = 'state_labels.csv'
state_labels = pd.read_csv(data_dir+state_file)

# Malaysia population by state (external data source)
state_pop_url="https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/static/population.csv"
state_pop_df=pd.read_csv(state_pop_url)

# Malaysia median income by state (external data source)
income_url = 'https://storage.googleapis.com/dosm-public-economy/salaries_state_sex.csv'
state_income_df=pd.read_csv(income_url)


In [31]:
income_url = 'https://storage.googleapis.com/dosm-public-economy/salaries_state_sex.csv'
state_income_df=pd.read_csv(income_url)
state_income_df.head()

,variable,variable_en,variable_bm,sex,year,mean,median,recipients
0,overall,Overall,Keseluruhan,overall,2010,1936.0,1500.0,7149200
1,overall,Overall,Keseluruhan,overall,2011,1959.0,1500.0,7634000
2,overall,Overall,Keseluruhan,overall,2012,2052.0,1566.0,7782900
3,overall,Overall,Keseluruhan,overall,2013,2186.0,1700.0,7903100
4,overall,Overall,Keseluruhan,overall,2014,2377.0,1800.0,8392400


In [32]:
print(train_tab_df.shape)
train_tab_df.columns

(14993, 24)


Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

### 1. Parse Pet Listing Image Metadata & Add to Tabular Dataframe

In [33]:
# set path to image metadata directory
image_metadata_path = 'train_metadata/' 

# lists to store each image metadata feature prior to EDA
vertex_xs = []
vertex_ys = []
bounding_confidences = []
bounding_importance_fracs = []
dominant_blues = []
dominant_greens = []
dominant_reds = []
dominant_pixel_fracs = []
dominant_scores = []
label_descriptions = []
label_scores = []
face_annotations = []
label_annotations = []
text_annotations = []

# lists of pet IDs without label annotations and/or any image metadata
petids_no_image_metadata = []
petids_no_label_annotation = []

# store each feature value for each pet ID in the main (tabular) df 
for pet_id in train_tab_df.PetID:
    try:
        # NOTE: selecting 1st image (when multiple images are available)
        with open(data_dir+image_metadata_path + pet_id + '-1.json', 'r') as f: 
            data = json.load(f)
        face_annotations.append(data.get('faceAnnotations', []))
        text_annotations.append(data.get('textAnnotations', []))
        vertex_x = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['x']
        vertex_xs.append(vertex_x)
        vertex_y = data['cropHintsAnnotation']['cropHints'][0]['boundingPoly']['vertices'][2]['y']
        vertex_ys.append(vertex_y)
        bounding_confidence = data['cropHintsAnnotation']['cropHints'][0]['confidence']
        bounding_confidences.append(bounding_confidence)
        bounding_importance_frac = data['cropHintsAnnotation']['cropHints'][0].get('importanceFraction', -1)
        bounding_importance_fracs.append(bounding_importance_frac)
        dominant_blue = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['blue']
        dominant_blues.append(dominant_blue)
        dominant_green = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['green']
        dominant_greens.append(dominant_green)
        dominant_red = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['color']['red']
        dominant_reds.append(dominant_red)
        dominant_pixel_frac = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['pixelFraction']
        dominant_pixel_fracs.append(dominant_pixel_frac)
        dominant_score = data['imagePropertiesAnnotation']['dominantColors']['colors'][0]['score']
        dominant_scores.append(dominant_score)
        # add label annotation data 
        if data.get('labelAnnotations'):
            label_annotations.append(data['labelAnnotations'])
            label_description = data['labelAnnotations'][0]['description']
            label_descriptions.append(label_description)
            label_score = data['labelAnnotations'][0]['score']
            label_scores.append(label_score)
        else:
            # append pet id to missing annotation list
            petids_no_label_annotation.append(pet_id)
            label_annotations.append(None)
            label_descriptions.append(None) # blank field = none
            label_scores.append(-1)
    except FileNotFoundError:
        # append pet id to missing annotation list
        petids_no_image_metadata.append(pet_id)
        vertex_xs.append(-1)
        vertex_ys.append(-1)
        bounding_confidences.append(-1)
        bounding_importance_fracs.append(-1)
        dominant_blues.append(-1)
        dominant_greens.append(-1)
        dominant_reds.append(-1)
        dominant_pixel_fracs.append(-1)
        dominant_scores.append(-1)
        label_annotations.append([None])
        label_descriptions.append(None) # blank field = none
        label_scores.append(-1)
        face_annotations.append([None])
        text_annotations.append([None])

print(f'Number of PetIDs missing label annotations: {len(petids_no_label_annotation)}')
print(f'Number of PetIDs missing image metadata: {len(petids_no_image_metadata)}')

# add new feature columns to main (tabular) dataframe
# dictionary of the new columns and lists to concat
image_data = {
    'vertex_xs': vertex_xs,
    'vertex_ys': vertex_ys,
     'bounding_confidences': bounding_confidences,
     'bounding_importance_fracs': bounding_importance_fracs,
     'dominant_blues': dominant_blues,
     'dominant_greens': dominant_greens,
     'dominant_reds': dominant_reds,
     'dominant_pixel_fracs': dominant_pixel_fracs,
     'dominant_scores': dominant_scores,
     'label_descriptions': label_descriptions,
     'label_scores': label_scores
}

#add new feature columns to main (tabular) dataframe
train_tab_df = pd.concat([
    train_tab_df,
    pd.DataFrame(image_data)],
    axis=1)
print(train_tab_df.shape)
print(train_tab_df.shape)

Number of PetIDs missing label annotations: 2
Number of PetIDs missing image metadata: 341
(14993, 35)
(14993, 35)


In [76]:
# for f in image_data:
#     print(f"{f} has length {len(image_data[f])}")
train_tab_df.shape[0] -341


14652

### 2. Parse Pet Listing Sentiment Analysis Data & Add to Dataframe

In [34]:
data = []

sentiment_path = './data/train_sentiment' 
json_files = [pos_json for pos_json in os.listdir(sentiment_path) if pos_json.endswith('.json')]

for js_file in json_files:
    with open(os.path.join(sentiment_path, js_file)) as f:
        data.append(json.load(f))

id_list = [f.split('.')[0] for f in json_files]  # assuming file name is petID
sent_magnitudes = [[sentence['sentiment']['magnitude'] for sentence in d['sentences']] if 'sentences' in d else None for d in data]
sent_scores = [[sentence['sentiment']['score'] for sentence in d['sentences']] if 'sentences' in d else None for d in data]
doc_magnitudes = [d['documentSentiment']['magnitude'] if 'documentSentiment' in d else None for d in data]
doc_scores = [d['documentSentiment']['score'] if 'documentSentiment' in d else None for d in data]
languages = [d['language'] if 'language' in d else None for d in data]
sent_texts = [[sentence['text']['content'] for sentence in d['sentences']] if 'sentences' in d else None for d in data]

sentiment_df = pd.DataFrame({
    'petID': id_list,
    'sent_scores': sent_scores,
    'sent_magnitudes': sent_magnitudes,
    'doc_scores': doc_scores,
    'doc_magnitudes': doc_magnitudes,
    'languages': languages,
    'text':sent_texts
})json_files

train_tab_df = pd.merge(train_tab_df, sentiment_df, how='left', left_on='PetID', right_on='petID')

print(train_tab_df.shape)
train_tab_df.head()

(14993, 42)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,dominant_scores,label_descriptions,label_scores,petID,sent_scores,sent_magnitudes,doc_scores,doc_magnitudes,languages,text
0,2,Nibble,3,299,0,1,1,7,0,1,...,0.302789,cat,0.990786,86e1089a3,"[0.8, 0.8, -0.2, 0, 0.1, 0.3]","[0.8, 0.8, 0.2, 0, 0.1, 0.3]",0.3,2.4,en,"[Nibble is a 3+ month old ball of cuteness., H..."
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,0.348178,cat,0.981269,6296e909a,"[0.1, -0.6]","[0.1, 0.6]",-0.2,0.7,en,[I just found it alone yesterday near my apart...
2,1,Brisco,1,307,0,1,2,7,0,2,...,0.333318,dog,0.960457,3422e4906,"[-0.9, 0, 0.9, 0.2, -0.1, 0.6, 0.7]","[0.9, 0, 0.9, 0.2, 0.1, 0.6, 0.7]",0.2,3.7,en,[Their pregnant mother was dumped by her irres...
3,1,Miko,4,307,0,2,1,2,0,2,...,0.136823,dog,0.978698,5842f1ff5,[0.9],[0.9],0.9,0.9,en,"[Good guard dog, very alert, active, obedience..."
4,1,Hunter,1,307,0,1,1,0,0,2,...,0.256168,dog,0.984346,850a43f90,"[0.7, 0.7, 0.6, 0.7, 0.8, 0]","[0.7, 0.7, 0.6, 0.7, 0.8, 0]",0.6,3.7,en,[This handsome yet cute boy is up for adoption...


In [78]:
len(json_files)

14442

### 3. Add external data features (Malaysia State Population & Median Income)

#### Review state labels for state IDs & state names in Petfinder tabular data

In [35]:
print(f"""Number of unique StateID values in train set:\n {len(train_tab_df.State.unique())}\n""")
print(f"""unique StateID values in train set:\n{train_tab_df.State.unique()}\n""")

Number of unique StateID values in train set:
 14

unique StateID values in train set:
[41326 41401 41330 41327 41361 41336 41324 41335 41325 41332 41345 41342
 41367 41415]



In [36]:
#rename 'State' col to 'StateID' for joining
train_tab_df.rename(columns={'State':'StateID'},inplace=True)
# check nulls in tabular data
print(len(train_tab_df[train_tab_df['StateID'].isnull()]))

0


#### Review state names for State IDs in Petfinder tabular data

In [37]:
print(f"""Number of unique State Names in Petfinder state labels:\n{len(state_labels.StateName.value_counts())}\n""")
print(f"""Unique State Names in Petfinder state labels:\n{state_labels.StateName.value_counts()}""")
# state_labels = state_labels.sort_values(by='StateName')
# state_labels.reset_index(inplace=True)

Number of unique State Names in Petfinder state labels:
15

Unique State Names in Petfinder state labels:
StateName
Johor              1
Kedah              1
Kelantan           1
Kuala Lumpur       1
Labuan             1
Melaka             1
Negeri Sembilan    1
Pahang             1
Perak              1
Perlis             1
Pulau Pinang       1
Sabah              1
Sarawak            1
Selangor           1
Terengganu         1
Name: count, dtype: int64


#### Review state income data (median income by state, 2020)

In [38]:
state_income_df.head()

,variable,variable_en,variable_bm,sex,year,mean,median,recipients
0,overall,Overall,Keseluruhan,overall,2010,1936.0,1500.0,7149200
1,overall,Overall,Keseluruhan,overall,2011,1959.0,1500.0,7634000
2,overall,Overall,Keseluruhan,overall,2012,2052.0,1566.0,7782900
3,overall,Overall,Keseluruhan,overall,2013,2186.0,1700.0,7903100
4,overall,Overall,Keseluruhan,overall,2014,2377.0,1800.0,8392400


In [39]:
# use 'variable_en' as state name
state_income_df.rename(columns={'variable_en':'state'},inplace=True)
state_income_df = state_income_df.sort_values(by='state')
print(f'Number of unique state names in state income data: {len(state_income_df.state.value_counts())}')
print(state_income_df.state.value_counts())

Number of unique state names in state income data: 17
state
Johor                36
Pulau Pinang         36
W.P. Labuan          36
W.P. Kuala Lumpur    36
Terengganu           36
Selangor             36
Sarawak              36
Sabah                36
Perlis               36
Kedah                36
Perak                36
Pahang               36
Overall              36
Negeri Sembilan      36
Melaka               36
Kelantan             36
W.P. Putrajaya       36
Name: count, dtype: int64


In [40]:
# subset 2020 data
state_income_df = state_income_df.loc[state_income_df['year']==2020]
# subset overall data (all genders)
state_income_df = state_income_df.loc[state_income_df['sex']=='overall'] 
# remove 'overall' (total) row
state_income_df = state_income_df.loc[state_income_df['state']!='Overall']

# remove 'W.P.' prefix to match state names in petfinder labels 
state_income_df['state'] = state_income_df['state'].str.replace('W.P. ','')
                                                          
state_income_df = state_income_df.loc[:,['state','median']]

# sort by English state names 
state_income_df = state_income_df.sort_values(by='state')

state_income_df.reset_index(drop=True,inplace=True)

print(f'Number of unique state names in clean state income data: {len(state_income_df.state.value_counts())}')
print(state_income_df.state.value_counts())
# 'Putrajaya' not present in other state dfs


Number of unique state names in clean state income data: 16
state
Johor              1
Kedah              1
Kelantan           1
Kuala Lumpur       1
Labuan             1
Melaka             1
Negeri Sembilan    1
Pahang             1
Perak              1
Perlis             1
Pulau Pinang       1
Putrajaya          1
Sabah              1
Sarawak            1
Selangor           1
Terengganu         1
Name: count, dtype: int64


#### Review state population data (total population by state, 2020)

In [41]:
state_pop_df.head()

,state,idxs,pop,pop_18,pop_60,pop_12,pop_5
0,Malaysia,0,32657100,23528200,3649000,3111400,3550500
1,Klang Valley,17,8417800,6181800,827600,719000,895100
2,Johor,1,3794200,2733400,445600,353900,409800
3,Kedah,2,2193600,1554800,282300,209700,254100
4,Kelantan,3,1928900,1258700,202100,211200,271700


In [42]:
print(f'Number of unique state names in state pop. data: {len(state_pop_df.state.value_counts())}')
print(state_pop_df.state.value_counts())

Number of unique state names in state pop. data: 18
state
Malaysia             1
Klang Valley         1
W.P. Labuan          1
W.P. Kuala Lumpur    1
Terengganu           1
Selangor             1
Sarawak              1
Sabah                1
Pulau Pinang         1
Perlis               1
Perak                1
Pahang               1
Negeri Sembilan      1
Melaka               1
Kelantan             1
Kedah                1
Johor                1
W.P. Putrajaya       1
Name: count, dtype: int64


In [43]:
# remove 'Malaysia' (total) row
state_pop_df = state_pop_df[state_pop_df['state'] != 'Malaysia'] 

# remove 'W.P.' prefix
state_pop_df['state'] = state_pop_df['state'].str.replace('W.P. ','')

state_pop_df = state_pop_df.sort_values(by='state')

print(f'Number of unique state names in cleaned state pop. data: {len(state_pop_df.state.value_counts())}')
print(state_pop_df.state.value_counts())
# 'Klang Valley' not present in other state df columns 

Number of unique state names in cleaned state pop. data: 17
state
Johor              1
Perak              1
Selangor           1
Sarawak            1
Sabah              1
Putrajaya          1
Pulau Pinang       1
Perlis             1
Pahang             1
Kedah              1
Negeri Sembilan    1
Melaka             1
Labuan             1
Kuala Lumpur       1
Klang Valley       1
Kelantan           1
Terengganu         1
Name: count, dtype: int64


In [44]:
# check difference between population df & income df states
print("state names in income df but not in population df:")
print (list(set(state_income_df.state.unique()) - set(state_pop_df.state.unique())))

state names in income df but not in population df:
['Pahang ', 'Pulau Pinang ']


In [45]:
# check difference between income df and population df: 
print("state names in population df but not in income df:")
print (list(set(state_pop_df.state.unique())- set(state_income_df.state.unique())))

state names in population df but not in income df:
['Pulau Pinang', 'Klang Valley', 'Pahang']


In [46]:
# remove trailing space after 'Pulau Pinang ' &  'Pahang ' in income df
state_income_df['state'] = state_income_df['state'].str.replace('Pulau Pinang ','Pulau Pinang')
state_income_df['state'] = state_income_df['state'].str.replace('Pahang ','Pahang')

In [47]:
# check that this worked
# check difference between population df & income df states
print("state names in income df but not in population df:")
print (list(set(state_income_df.state.unique()) - set(state_pop_df.state.unique())))

print("state names in population df but not in income df:")
print (list(set(state_pop_df.state.unique())- set(state_income_df.state.unique())))

state names in income df but not in population df:
[]
state names in population df but not in income df:
['Klang Valley']


In [48]:
state_pop_df.reset_index(drop=True,inplace=True)
# merge external state data
comb_state_df = pd.merge(state_pop_df,state_income_df,how='left',on='state')
# select relevant columns 
comb_state_df = comb_state_df.loc[:,['state','pop','median']]

print(f'Number of unique state names in combined state data: {len(comb_state_df.state.value_counts())}')
print(f'Number of unique state names in P.F. State Labels: {len(state_labels.StateName.value_counts())}')

Number of unique state names in combined state data: 17
Number of unique state names in P.F. State Labels: 15


In [49]:
# check discrepancies in state names between combined income/pop data and PF labels
print("State names in median/income df but not in Petfinder labels:")
print (list(set(comb_state_df.state.unique())- set(state_labels.StateName.unique())))

print("State names in Petfinder labels but not in median/income df:")
print (list(set(state_labels.StateName.unique()) - set(comb_state_df.state.unique())))

State names in median/income df but not in Petfinder labels:
['Putrajaya', 'Klang Valley']
State names in Petfinder labels but not in median/income df:
[]


In [50]:
# rename columns for clarity 
comb_state_df.rename(columns={"state":"StateName",
                              "pop":"state_population",
                               "median":"median_state_income"}, 
                      inplace=True)

# merge with petfinder state labels 
state_labels = pd.merge(state_labels,comb_state_df,on='StateName')
state_labels

,StateID,StateName,state_population,median_state_income
0,41336,Johor,3794200,2124.0
1,41325,Kedah,2193600,1474.0
2,41367,Kelantan,1928900,1343.0
3,41401,Kuala Lumpur,1746600,3037.0
4,41415,Labuan,100100,2130.0
5,41324,Melaka,937800,2120.0
6,41332,Negeri Sembilan,1128900,2062.0
7,41335,Pahang,1684700,1753.0
8,41330,Perak,2509000,1629.0
9,41380,Perlis,255500,1571.0


#### Join petfinder state labels with combined external state data

In [51]:
# join petfinder state labels with external state data 
train_tab_df = pd.merge(train_tab_df,state_labels,how='left', on='StateID')
#train_tab_df= train_tab_df.set_index('StateID').join(state_labels.set_index('StateID'))
print(train_tab_df.shape)
print(train_tab_df.columns)

(14993, 45)
Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'StateID', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed',
       'vertex_xs', 'vertex_ys', 'bounding_confidences',
       'bounding_importance_fracs', 'dominant_blues', 'dominant_greens',
       'dominant_reds', 'dominant_pixel_fracs', 'dominant_scores',
       'label_descriptions', 'label_scores', 'petID', 'sent_scores',
       'sent_magnitudes', 'doc_scores', 'doc_magnitudes', 'languages', 'text',
       'StateName', 'state_population', 'median_state_income'],
      dtype='object')


In [52]:
train_tab_df.median_state_income.value_counts()

median_state_income
2725.0    8714
3037.0    3845
2082.0     843
2124.0     507
1629.0     420
2062.0     253
2120.0     137
1474.0     110
1753.0      85
1514.0      26
1716.0      22
1343.0      15
1593.0      13
2130.0       3
Name: count, dtype: int64

### -------begin Bailey added code section-------

# Split the combined data into test and train (80/20)

#### Split data into test/train

In [53]:
train_tab_df.AdoptionSpeed.value_counts(normalize=True)

AdoptionSpeed
4    0.279931
2    0.269259
3    0.217368
1    0.206096
0    0.027346
Name: proportion, dtype: float64

In [54]:
# split the data into test and train (split training 80% so that 25% of it can be used for validation)

# Original version:
train, test = train_test_split(train_tab_df, test_size=0.2, random_state=42)

# Stratified version:
#train, test = train_test_split(train_tab_df, test_size=0.2,random_state=42, 
#                               stratify = train_tab_df.AdoptionSpeed) # stratify to balance labels in train/test sets


In [55]:
# review shape of data
print('Shape train: ', train.shape, '\nShape test: ', test.shape)
      #,'\nShape validation: ', validation.shape)

Shape train:  (11994, 45) 
Shape test:  (2999, 45)


In [56]:
train.AdoptionSpeed.value_counts(normalize=True)

AdoptionSpeed
4    0.280474
2    0.269385
3    0.218276
1    0.205353
0    0.026513
Name: proportion, dtype: float64

#### Write data to respective files
Note: The files will split the same way each time as long as you do not change the "random_state" in the train_test_split function.


In [57]:
# train
train.to_csv('split_train.csv')

In [58]:
# test 
test.to_csv('split_test.csv')